In [1]:
import csv
import copy
import os
import random
import boto3
import zipfile
import geopandas as gpd

In [2]:
cri_indicator_list = [
'society_vulnerable_populations',
'built_tranportation_infrastructure',
'society_social_services',
'society_economic_health',
'governance_emergency_response',
'governance_personal_preparedness',
'governance_community_preparedness',
'governance_natural_resource_conservation',
'natural_ecosystem_type_condition',
'natural_agricultural_productivity',
'built_communication_infrastructure',
'built_utility_infrastructure',
'built_housing_vacancy_quality',
'climate_wildfire_exposure',
'climate_wildfire_loss',
'climate_flood_exposure',
'climate_flood_loss',
'climate_drought_exposure',
'climate_drought_loss',
'climate_heat_exposure',
'climate_heat_loss',
'climate_slr_exposure',
'climate_slr_loss'
]

In [3]:
def dummy_data_setup(your_indicators):
    '''
    Uses a template .csv file containing all California census tract codes and creates a replicate .csv file for each indicator listed. 
    Random values between 0-1 are generated next to each census tract within each unique .csv file.
    
    Note:
    This function assumes users have configured the AWS CLI such that their access key / secret key pair are stored in ~/.aws/credentials.
    See https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html for guidance.
    
    Parameters
    ----------
    your_indicators: list
              List of indicator names
    '''

    # Read in CA census tiger file
    print('Reading in census data')
    census_shp_dir = "s3://ca-climate-index/0_map_data/2021_tiger_census_tract/2021_ca_tract/"
    ca_boundaries = gpd.read_file(census_shp_dir)
    print('Complete')
    print('')

   # Selecting the 'GEOID' column
    census_tract_geoid = ca_boundaries[['GEOID']]

    # Writing the selected columns to a CSV file
    census_tract_geoid.to_csv('dummy_dataset_template.csv', index=False)

    # Making a CSV template
    template_header = ['GEOID', your_indicators[0]]
    
    # Read the template CSV and store its rows
    template_rows = []
    with open('dummy_dataset_template.csv', 'r') as template_file:
        template_reader = csv.reader(template_file)
        template_rows = list(template_reader)

    # Create a folder to store the new CSV files
    output_folder = 'dummy_dataset'
    os.makedirs(output_folder, exist_ok=True)

    # Loop through each indicator in the list
    for indicator in your_indicators:
        # Update the header with the current indicator name
        template_header[1] = indicator

        # Create a new CSV file with the indicator name
        filename = os.path.join(output_folder, f'DUMMY_{indicator}_indicator.csv')

        # Open the CSV file in write mode
        with open(filename, 'w', newline='') as csvfile:
            # Create a CSV writer object
            csvwriter = csv.writer(csvfile)

            # Write the updated template header to the new CSV file
            csvwriter.writerow(template_header)

            # Write the template rows to the new CSV file
            for template_row in template_rows[1:]:  # Skip the header row
                new_row = copy.deepcopy(template_row)

                # Randomly generate 0-1 value in the second column
                new_row.append(f'{random.uniform(0, 1):.3f}')  # Append the random value to the end
                
                csvwriter.writerow(new_row)

            print(f'{filename} created successfully.')


    # Zip the folder
    zip_filename = 'dummy_dataset.zip'
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for root, _, files in os.walk(output_folder):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, output_folder)
                zipf.write(file_path, arcname)

    print('')
    print('Uploading data to AWS')
    # Set-up for AWS  
    s3_client = boto3.client('s3')  
    bucket_name = 'ca-climate-index'  

    with open(zip_filename, 'rb') as data:
        s3_client.upload_fileobj(data, bucket_name, f"{'3_fair_data/dummy_data'}/{zip_filename}")
    print(f"Zip file {zip_filename} uploaded to AWS S3.")

dummy_data_setup(cri_indicator_list)


Reading in census data
Complete

dummy_dataset\DUMMY_society_vulnerable_populations_indicator.csv created successfully.
dummy_dataset\DUMMY_built_tranportation_infrastructure_indicator.csv created successfully.
dummy_dataset\DUMMY_society_social_services_indicator.csv created successfully.
dummy_dataset\DUMMY_society_economic_health_indicator.csv created successfully.
dummy_dataset\DUMMY_governance_emergency_response_indicator.csv created successfully.
dummy_dataset\DUMMY_governance_personal_preparedness_indicator.csv created successfully.
dummy_dataset\DUMMY_governance_community_preparedness_indicator.csv created successfully.
dummy_dataset\DUMMY_governance_natural_resource_conservation_indicator.csv created successfully.
dummy_dataset\DUMMY_natural_ecosystem_type_condition_indicator.csv created successfully.
dummy_dataset\DUMMY_natural_agricultural_productivity_indicator.csv created successfully.
dummy_dataset\DUMMY_built_communication_infrastructure_indicator.csv created successfull